#**Aplikasi Marketplace TokoPaedi**

##**Fitur**

1. Registrasi User
2. Login User
3. Menampilkan Semua Produk
4. Beli Produk
5. Menampilkan Transaksi Pembelian
6. Menampilkan Produk saya
7. Tambah Produk
8. Menampilkan Transaksi toko
9. Menampilkan Grafik Penjualan
10. Isi Saldo

##**Library**


1.   Tabulate : untuk print Data kedalam bentuk tabel
2.   Matplotlib : Visualisasi Data
3.   JSON : untuk handling / parse Data JSON
4.   OS : library bawaan untuk check file dan eksekusi shell
5.   Ipython : Clear / hapus log output di jupyter notebook





In [ ]:
import tabulate
import matplotlib.pyplot as plt
import json
import os
from IPython.display import clear_output 

variabel database digunakan untuk menyimpan dan mengakses data , dari aplikasi yang dibuat

In [ ]:
database = {}

fungsi baris dibawah adalah
mengecek apakah ada file database.json, jika ada akan dijadikan sebagai value dari database

In [ ]:
if os.path.isfile("database.json"):
    with open("database.json", "r") as file:
        database = json.load(file)
else:
    database = {
        "pengguna": [],
        "barang": [],
        "transaksi": []
    }

**function clearOutput** : digunakan untuk menghapus log / output agar tidak timpang tindih saat aplikasi berjalan

function ini dapat menyesuaikan environtment yang digunakan

In [ ]:
def clearOutput():
    if os.name == "nt": os.system("cls")
    elif os.name == "posix": clear_output(wait=True)
    else : os.system("clear")

**function checkUsername** : digunakan untuk mengecek apakah username yang dilemparkan dari parameter sudah ada di database, dengan melakukan pengecekan atau looping

In [ ]:
def checkUsername(username):
    for pengguna in database["pengguna"]:
        if(pengguna["username"] == username):
            return True
    return False

**function registrasiUser** : digunakan untuk mendaftarkan atau menambahkan user kedalam database

In [ ]:
def registrasiUser(username, password, nama):
    if not checkUsername(username):
        database["pengguna"].append({
            "nama": nama,
            "username": username,
            "password": password,
            "saldo": 0
        })
        return True
    else:
        return False

**function loginUser** : digunakan untuk melakukan authentikasi / verifikasi apakah username dan password yang dilemparkan dari parameter ada yang sesuai dengan yang ada di database

In [ ]:
def loginUser(username, password):
    for index, pengguna in enumerate(database["pengguna"]):
        if pengguna["username"] == username and pengguna["password"] == password:
            return index
    return False

**function tambahBarang** : digunakan untuk menambahkan barang atau produk kedalam database

In [ ]:
def tambahBarang(namaProduk, hargaProduk, stokProduk, username):
    if checkUsername(username):
        database["barang"].append({
            "idProduk": len(database["barang"]) + 1,
            "namaProduk": namaProduk,
            "hargaProduk": int(hargaProduk),
            "stokProduk": int(stokProduk),
            "pemilik": username
        })
        return True
    return False

**function tampilSemuaProduk** : digunakan untuk menampilkan semua produk berdasarkan parameter action yang diberikan

jika parameter yang diberikan = buy maka akan 
menampilkan produk yang dapat dibeli

jika parameter yang diberikan = sell maka akan menampilkan produk yang dijual dari parameter username

list produk akan ditampilkan dalam bentuk tabel menggunakan function tabulate dari library tabulate

In [ ]:
def tampilSemuaProduk(username, action):
    produkTampil = []

    if action == "buy":
        for produk in database["barang"]:
            if produk["pemilik"] != username:
                produkTampil.append(produk.values())
    elif action == "sell":
        for produk in database["barang"]:
            if produk["pemilik"] == username:
                produkTampil.append(produk.values())

    print(tabulate.tabulate(produkTampil, headers=[
          "ID", "Nama Produk", "Harga Produk", "Stok Produk", "Pemilik"]))

**function getSaldo** : digunakan untuk mengakses / mendapatkan informasi saldo user dari database

In [ ]:
def getSaldo(username):
    for index, pengguna in enumerate(database["pengguna"]):
        if pengguna["username"] == username:
            return database["pengguna"][index]["saldo"]
    return False

**function getIndexPengguna** : digunakan untuk mendapatkan Index dari array pengguna yang sesuai dengan parameter user

In [ ]:
def getIndexPengguna(username):
    for index, pengguna in enumerate(database["pengguna"]):
        if pengguna["username"] == username:
            return index
    return False

**function transaksi** digunakan untuk melakukan transaksi

didalam fungsi transaksi terdapat beberapa validasi antara lain :

1.   pemilik produk tidak boleh melakukan pembelian pada produknya sendiri 
2.   saldo pembeli harus sama atau melebihi total transaksi
3.   stok barang harus sama atau melebihi total barang yang ingin dibeli


setelah berhasil dilakukan validasi akan dilakukan : 
1. Update saldo pembeli dan penjual
2. Update stok barang
3. Penambahan data transaksi ke dalam database


In [ ]:
def transaksi(username, idProduk, jumlahBeli):
    for index, produk in enumerate(database["barang"]):
        if produk["idProduk"] == int(idProduk):
            if produk["pemilik"] == username:
                print("Tidak boleh membeli produk milik anda sendiri")
                return False

            if(getSaldo(username) >= produk["hargaProduk"] * int(jumlahBeli)):
                if(produk["stokProduk"] >= int(jumlahBeli)):
                    # Update Saldo
                    database["pengguna"][getIndexPengguna(
                        username)]["saldo"] -= produk["hargaProduk"] * int(jumlahBeli)
                    database["pengguna"][getIndexPengguna(
                        produk["pemilik"])]["saldo"] += produk["hargaProduk"] * int(jumlahBeli)

                    # Update Stok
                    database["barang"][index]["stokProduk"] -= int(jumlahBeli)

                    # Menambahkan Transaksi
                    database["transaksi"].append({
                        "idTransaksi": len(database["transaksi"]) + 1,
                        "idProduk": int(idProduk),
                        "namaProduk": produk["namaProduk"],
                        "jumlah": int(jumlahBeli),
                        "totalTransaksi": produk["hargaProduk"] * int(jumlahBeli),
                        "buyer": username,
                        "seller": produk["pemilik"]
                    })
                    return True
                else:
                    print("Stok tidak mencukupi")
                    return False
            else:
                print("Saldo tidak mencukupi")
                return False
    print("Tidak ada barang dengan id tersebut")
    return False

**function tambahSaldo** : digunakan untuk menambahkan saldo kedalam database user

In [ ]:
def tambahSaldo(username, jumlahDeposit):
    index = getIndexPengguna(username)
    if type(index) == int:
        database["pengguna"][index]["saldo"] += int(jumlahDeposit)
        return True
    return False

**function getTransaksi** : digunakan untuk menampilkan semua transaksi berdasarkan parameter action yang diberikan

jika parameter yang diberikan = buy maka akan 
menampilkan histori transaksi pembelian yang pernah dilakukan

jika parameter yang diberikan = sell maka akan menampilkan histori transaksi penjualan yang pernah dilakukan

list histori transaksi akan ditampilkan dalam bentuk tabel menggunakan function tabulate dari library tabulate

In [ ]:
def getTransaksi(username, action):
    transaksiTampil = []

    if action == "buy":
        for transaksi in database["transaksi"]:
            if transaksi["buyer"] == username:
                transaksiTampil.append(transaksi.values())

    elif action == "sell":
        for transaksi in database["transaksi"]:
            if transaksi["seller"] == username:
                transaksiTampil.append(transaksi.values())

    else:
        return False

    print(tabulate.tabulate(transaksiTampil, headers=[
          "ID", "ID Produk", "Nama Produk", "Jumlah", "Total Transaksi", "Buyer", "Seller"]))

**function grafikPenjualan** : digunakan untuk menampilkan grafik penjualan dalam bentuk bar plot, agar memudahkan penjual dalam melihat transaksi produk yang paling laku

dalam membuat visualisasi data kami menggunakan library dari matplotlib

In [ ]:
def grafikPenjualan(username):
    namaProduk = tuple()
    hargaPenjualan = []

    for transaksi in database["transaksi"]:
        if transaksi["seller"] == username:
            if transaksi["namaProduk"] not in namaProduk:
                namaProduk += (transaksi["namaProduk"],)
                hargaPenjualan += (transaksi["totalTransaksi"],)
            else:
                hargaPenjualan[namaProduk.index(
                    transaksi["namaProduk"])] += transaksi["totalTransaksi"]

    plt.bar(list(namaProduk), list(hargaPenjualan))
    plt.title("Grafik Penjualan")
    plt.xlabel("Nama Produk")
    plt.ylabel("Total Penjualan")
    plt.show()


kode dibawah digunakan sebagai state / acuan dalam menampilkan menu

1. jika status isLoggedIn = True maka akan dianggap sudah berhasil login dan masuk kedalam menu User
2. jika status isLoggedIn = False maka akan diarahkan ke menu authentikasi untuk melakukan registrasi maupun login

In [ ]:
state = {
    "isLoggedIn": False,
    "username": None
}

Kode dibawah digunakan untuk menjalankan aplikasi, berisi logic logic untuk memanggil function function yang sudah dibuat sebelumnya

In [ ]:
while True:
    clearOutput()
    if not state["isLoggedIn"]:
        print("Selamat Datang di TokoPaedi")
        print("Silahkan pilih menu berikut : ")
        print("1. Daftar")
        print("2. Login")
        print("3. Keluar")
        aksiPertama = input("Masukan pilihan anda : ")
        if aksiPertama == "1":
            username = input("Masukan username : ")
            if not username.isspace() and not username.isalpha():
                print("Username tidak boleh mengandung spasi dan harus berupa huruf")
                continue
            password = input("Masukan password : ")
            nama = input("Masukan nama : ")
            registrasi = registrasiUser(username, password, nama)
            if registrasi == False:
                print("Gagal Daftar! Username sudah ada")
                continue
            else:
                print("Registrasi berhasil")
                state = {
                    "isLoggedIn": True,
                    "username": username
                }
                continue
        elif aksiPertama == "2":
            username = input("Masukan username : ")
            password = input("Masukan password : ")
            login = loginUser(username, password)
            if type(login) == bool:
                print("Gagal Login ! Username atau password salah")
                continue
            else:
                state = {
                    "isLoggedIn": True,
                    "username": username
                }
                continue
        elif aksiPertama == "3":
            with open('database.json', 'w') as outfile:
                json.dump(database, outfile, indent=4)
            print("Terima kasih telah menggunakan layanan kami")
            break
        else:
            print("Menu yang anda pilih tidak tersedia")
    else:
        username = state["username"]
        saldo = database["pengguna"][getIndexPengguna(username)]["saldo"]
        nama = database["pengguna"][getIndexPengguna(username)]["nama"]

        print(f"Selamat Datang {nama},\ndi TokoPaedi")
        print(f"Saldo anda saat ini : {saldo}\n")
        print("Silahkan pilih menu berikut : ")
        print("1. Tampilkan Semua Produk")
        print("2. Beli Produk")
        print("3. Lihat Transaksi Pembelian")
        print("4. Tampilkan Produk saya")
        print("5. Tambah Produk")
        print("6. Lihat Transaksi toko")
        print("7. Tampilkan Grafik Penjualan")
        print("8. Isi Saldo")
        print("9. Logout")
        aksiKedua = input("Masukan pilihan anda : ")
        if aksiKedua == "1":
            tampilSemuaProduk(username, "buy")
        elif aksiKedua == "2":
            tampilSemuaProduk(username, "buy")
            idProduk = input("Masukan id produk yang ingin anda beli : ")
            jumlahProduk = input(
                "Masukan jumlah produk yang ingin anda beli : ")
            Transaksi = transaksi(username, idProduk, jumlahProduk)
            if Transaksi == False:
                print("Transaksi gagal")
            else:
                print("Transaksi berhasil")
        elif aksiKedua == "3":
            getTransaksi(username, "buy")
        elif aksiKedua == "4":
            tampilSemuaProduk(username, "sell")
        elif aksiKedua == "5":
            namaProduk = input("Masukan nama produk : ")
            hargaProduk = input("Masukan harga produk : ")
            if not hargaProduk.isdigit():
                print("Harga produk harus berupa angka")
                continue
            stokProduk = input("Masukan stok produk : ")
            if not stokProduk.isdigit():
                print("Stok produk harus berupa angka")
                continue
            tambahProduk = tambahBarang(
                namaProduk, hargaProduk, stokProduk, username)
            if tambahProduk == False:
                print("Gagal Tambah Produk")
            else:
                print("Tambah Produk Berhasil")
        elif aksiKedua == "6":
            getTransaksi(username, "sell")
        elif aksiKedua == "7":
            grafikPenjualan(username)
        elif aksiKedua == "8":
            jumlahDeposit = input("Masukan jumlah deposit : ")
            if not jumlahDeposit.isdigit():
                print("Jumlah deposit harus berupa angka")
                continue
            deposit = tambahSaldo(username, jumlahDeposit)
            if deposit == False:
                print("Gagal Tambah Saldo")
            else:
                print("Tambah Saldo Berhasil")
        elif aksiKedua == "9":
            state = {
                "isLoggedIn": False,
                "username": None
            }
            continue
        else:
            print("Menu yang anda pilih tidak tersedia")
    input("Tekan enter untuk melanjutkan...")

Selamat Datang di TokoPaedi
Silahkan pilih menu berikut : 
1. Daftar
2. Login
3. Keluar
Masukan pilihan anda : 3
Terima kasih telah menggunakan layanan kami
